Deep contextualized word representation has drawn wide attention because of state-of-the-art performances in downstream tasks. Contextualized embeddings can capture not only word-level information but also multi-sense information, thus improving the results in sentiment analysis, SQuad and etc. However, the language adopted in the [Elmo](https://allennlp.org/elmo) model were biLSTMs which contained a huge number of parameters, it was less likely for small labs to train and run such experiments.


In this project, we intend to make use of CNN language model in learning efficient word representations for sentiment analysis. We train a language model based on [Gated CNN architecture](https://arxiv.org/abs/1612.08083) proposed by Yann Daulphin, then do sentiment analysis with embeddings generated by the language model.

The language model training dataset is 1-billion-word-language.

In [1]:
import tensorflow as tf
import os
import time
import numpy as np
from model import GatedCharCNN
from bilm.training import load_options_latest_checkpoint, load_vocab
from bilm.data import Batcher, BidirectionalLMDataset
from data_utils import data_helper
from conf_utils import *

/home/nlp/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## Initialize the configuration and prepare data batches

In [2]:
#Load the words
vocab_file = 'data/vocab-2016-09-10.txt'
vocab = load_vocab(vocab_file, 50)

In [3]:
class config:a
    vocab_size = vocab.size
    embedding_size = 100
    filter_size = 64
    num_layers = 3
    block_size = 3
    filter_h = 5
    context_size = 20
    text_size = context_size
    batch_size = 32
    epochs = 5
    num_sampled = 64
    learning_rate = 0.0001
    momentum = 0.99
    grad_clip = 0.1
    num_batches = 0
    ckpt_path = 'ckpt'
    summary_path = 'logs'
    #data_dir = "data/texts/reviews/movie_reviews"
    data_dir = "data/1-billion-word-language-modeling-benchmark-r13output/training-monolingual.tokenized.shuffled"

In [4]:
#Initialize configuration files
conf = prepare_conf(config)

In [5]:
train_prefix = 'data/1-billion-word-language-modeling-ben\
chmark-r13output/training-monolingual.tokenized.shuffled/*'

In [6]:
data = BidirectionalLMDataset(train_prefix, vocab, test=False,
                                      shuffle_on_load=True)

Found 99 shards at data/1-billion-word-language-modeling-benchmark-r13output/training-monolingual.tokenized.shuffled/*
Loading data from: data/1-billion-word-language-modeling-benchmark-r13output/training-monolingual.tokenized.shuffled/news.en-00020-of-00100
Loaded 305446 sentences.
Finished loading
Found 99 shards at data/1-billion-word-language-modeling-benchmark-r13output/training-monolingual.tokenized.shuffled/*
Loading data from: data/1-billion-word-language-modeling-benchmark-r13output/training-monolingual.tokenized.shuffled/news.en-00086-of-00100
Loaded 305744 sentences.
Finished loading


In [7]:
data_gen = data.iter_batches(conf.batch_size * 1, conf.text_size)

In [8]:
x = next(data_gen)

In [9]:
x.keys()

dict_keys(['token_ids', 'tokens_characters', 'next_token_id', 'token_ids_reverse', 'tokens_characters_reverse', 'next_token_id_reverse'])

In [10]:
x['token_ids'].shape

(32, 20)

In [10]:
x['next_token_id'].shape

(32, 20)

In [11]:
x.keys()

dict_keys(['token_ids', 'tokens_characters', 'next_token_id', 'token_ids_reverse', 'tokens_characters_reverse', 'next_token_id_reverse'])

## Train a CharCNN-based language model

Note the inputs are transformed into chars of words, so as to make use of subword information.

In [5]:
#Create a language model
#Note we need to save the models for subsequent tasks
model = GatedCharCNN(conf)
saver = tf.train.Saver(tf.trainable_variables())
print("Started Model Training...")

Started Model Training...


In [9]:
#from RNN import RNN
#model = RNN(conf)
#saver = tf.train.Saver(tf.trainable_variables())

In [12]:
model.y

<tf.Tensor 'Reshape_3:0' shape=(?, 1) dtype=int32>

In [13]:
conf.ckpt_file

'ckpt/vocab793471_embed100_filters64_batch32_layers3_block3_fdim5/model.ckpt'

In [14]:
batch_idx = 0
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    summary_writer = tf.summary.FileWriter(conf.summary_path, graph=sess.graph)

    if os.path.exists(conf.ckpt_file+'.index'):
        saver.restore(sess, conf.ckpt_file)
        print("Model Restored")

    for i in np.arange(conf.epochs):
        start = time.time()
        for j in np.arange(10000):
        #for j in np.arange(21):
            x = next(data_gen)
            inputs, labels = x['tokens_characters'], x['next_token_id']
            labels = labels.reshape(-1, 1)
            _, l = sess.run([model.optimizer, model.loss], feed_dict={model.X:inputs, model.y:labels})
            if j%200 == 0:
                print(l)
        end = time.time()
        print("Epoch: %.2f, Time: %.2f,  Loss: %.2f"%(i, end-start, l))

        if i % 2 == 0:
            perp = sess.run(model.perplexity, feed_dict={model.X:inputs, model.y:labels})
            print("Perplexity: %.2f"%perp)
            saver.save(sess, conf.ckpt_file)

        summaries = sess.run(model.merged_summary_op, feed_dict={model.X:inputs, model.y:labels})
        summary_writer.add_summary(summaries, i)

INFO:tensorflow:Restoring parameters from ckpt/vocab793471_embed100_filters64_batch32_layers3_block3_fdim5/model.ckpt
Model Restored
5.6052504


KeyboardInterrupt: 

## Train a RNN model

## Sentiment Analysis

In this part, we need to use other datasets for sentiment analysis, like the one of SemEval2013.

In [6]:
file_train = 'data/semeval/downloaded.tsv'
file_dev = 'data/semeval/dev_downloaded.tsv'
file_test = 'data/semeval/test.txt'
with open(file_train) as f:
    tweets_train = f.readlines()
with open(file_dev) as f:
    tweets_dev = f.readlines()
with open(file_test) as f:
    tweets_test = f.readlines()
    


#Filter empty tweets
def is_available(text):
    if 'Not Available' in text:
        return False
    if '\t"objective' in text:
        return False
    if '\t"neutral' in text:
        return False
    if '\tobjective' in text:
        return False
    if '\tneutral' in text:
        return False
    return True


In [7]:
tweets_train = list(filter(is_available, tweets_train))
tweets_dev = list(filter(is_available, tweets_dev))
tweets_test = list(filter(is_available, tweets_test))

In [8]:
tweets_train = [item.split('\t') for item in tweets_train]
tweets_dev = [item.split('\t') for item in tweets_dev]
tweets_test = [item.split('\t') for item in tweets_test]
_, _, y_train, text_train = list(zip(*tweets_train))
_, _, y_dev, text_dev = list(zip(*tweets_dev))
_, _, y_test, text_test = list(zip(*tweets_test))


In [9]:
text_train, y_train = list(text_train), list(y_train)
text_dev, y_dev = list(text_dev), list(y_dev)
text_test, y_test = list(text_test), list(y_test)
y_test = ['"' + item + '"' for item in y_test]

In [10]:
#Encode the labels to numbers
from sklearn import preprocessing
label_encode = preprocessing.LabelEncoder()  # 建立模型
y_train = label_encode.fit_transform(y_train)
y_dev = label_encode.transform(y_dev)
y_test = label_encode.transform(y_test)

## Twitter Data Preprocessing

In [11]:
from sents_handler import generate_char_samples
import numpy as np

train_gs = generate_char_samples(np.array(text_train), np.array(y_train), vocab_file, 20, True)
#sent_vecs, sent_labels, lengths = train_gs.generate(32)

In [12]:
sent_vecs, sent_labels, lengths = train_gs.generate(32)

In [13]:
sent_vecs.shape

(32, 22, 50)

Get the contextualized representation.

In [14]:
sess_lm = tf.Session()
saver.restore(sess_lm, conf.ckpt_file)
def sent2vec(inputs, sess):
    '''Get word representations'''
    #Get the contextualized representation
    #train_gs = generate_samples(np.array(text_train), np.array(y_train), word_to_idx, 20, False)
    #sent_vecs, sent_labels, lengths = train_gs.generate(32)
    out_layer = sess.run(model.out_layer, feed_dict={model.X:inputs})
    return out_layer
    

INFO:tensorflow:Restoring parameters from ckpt/vocab793471_embed100_filters64_batch32_layers3_block3_fdim5/model.ckpt


In [15]:
sent_vecs, sent_labels, lengths = train_gs.generate(32)
out_layer = sent2vec(sent_vecs, sess_lm)

In [16]:
out_layer.shape

(32, 22, 100)

In [17]:
max_word_len = out_layer.shape[1]
from sentiment_analysis import CNN_Model_Pretrained_Emb
class trainConfig:
    max_doc_len = max_word_len
    label_size = 2
    embed_size = 100
    hidden_size = 250
    batch_size = 32
    layer_size = 2
    
class testConfig:
    max_doc_len = max_word_len
    label_size = 2
    embed_size = 100
    hidden_size = 250
    batch_size = 32
    layer_size = 2
    
class singleConfig:
    max_doc_len = max_word_len
    label_size = 2
    embed_size = 100
    hidden_size = 250#hidden size for hidden state of rnn
    batch_size = 1
    layer_size = 2


In [18]:
import tensorflow as tf
graph_cnn = tf.Graph()
#Create models for training and testing data
with graph_cnn.as_default():
    initializer = tf.random_uniform_initializer(-0.02, 0.02)
    with tf.name_scope('train'):
        #Set different models for different buckets
        with tf.variable_scope("Model", reuse=None, initializer=initializer):
            train_model = CNN_Model_Pretrained_Emb(trainConfig)
            saver_sent=tf.train.Saver()
    with tf.name_scope('test'):
        #Set different models for different buckets
        with tf.variable_scope("Model", reuse=True, initializer=initializer):
            test_model = CNN_Model_Pretrained_Emb(testConfig, False)
            single_model = CNN_Model_Pretrained_Emb(singleConfig, False)



Model Initialized!
Model Initialized!
Model Initialized!


In [19]:
train_chunk_num  = int(len(text_train)/trainConfig.batch_size)
test_chunk_num = int(len(text_test)/testConfig.batch_size)
remain_num = len(text_test) - testConfig.batch_size*test_chunk_num
remain_num

21

In [20]:
import time, os
epochs = 5
#train_chunk_num = 10
file = "ckpt_cnn_pretrained_emb/cnn.ckpt"
with tf.Session(graph=graph_cnn) as sess:
    #Initialize parameters
    init = tf.global_variables_initializer()
    if not os.path.exists("ckpt_cnn_pretrained_emb"):
        os.mkdir('ckpt_cnn_pretrained_emb')
    if os.path.exists("ckpt_cnn_pretrained_emb/cnn.ckpt.index"):
        saver_sent.restore(sess, file)
    else:
        sess.run(init)
    start_time = time.time()
    for m in range(epochs):
        for i in range(train_chunk_num):
            #sess.run(tf.assign(learning_rate, 0.002*((0.98)**m)))
            x, y, lengths = train_gs.generate(trainConfig.batch_size)
            x = sent2vec(x, sess_lm)
            feed_dict = {train_model.x:x, train_model.y:y, train_model.lengths:lengths}
            l, _ = sess.run([train_model.cost, train_model.optimize], feed_dict=feed_dict)
            if i%100 == 0:
                print('Loss:', round(l, 4))
        end_time = time.time()
        print('Epoch', m, 'time:{:.2f}'.format(end_time - start_time))
        if m%2 == 0:
            saver_sent.save(sess,'ckpt_cnn_pretrained_emb/cnn.ckpt')

Loss: 0.6876
Loss: 0.53
Epoch 0 time:21.78
Loss: 0.6323
Loss: 0.5048
Epoch 1 time:43.27
Loss: 0.6656
Loss: 0.5963
Epoch 2 time:64.96
Loss: 0.5998


KeyboardInterrupt: 

In [22]:
#Calculate Testing Accuracy
with tf.Session(graph=graph_cnn) as sess:
    print('Testing...')
    count = 0
    #saver = tf.train.import_meta_graph('ckpt_cnn/cnn.ckpt.meta')
    saver_sent.restore(sess,tf.train.latest_checkpoint('ckpt_cnn_pretrained_emb/'))
    print('Parameters restored')
    start_time = time.time()
    test_gs = generate_char_samples(np.array(text_test), np.array(y_test),vocab_file, 20, False)
    for _ in range(test_chunk_num):
        #Traverse each data
        x, y, lengths = test_gs.generate(testConfig.batch_size)
        x = sent2vec(x, sess_lm)
        feed_dict = {test_model.x:x, test_model.y:y, test_model.lengths:lengths}
        n = sess.run(test_model.correct_num, feed_dict=feed_dict)
        count += np.sum(n)
    for i in range(remain_num):
        #Traverse each data
        x, y, lengths = test_gs.generate(1)
        #print(i, len(x))
        x = sent2vec(x, sess_lm)
        feed_dict = {single_model.x:x, single_model.y:y, 
                     single_model.lengths:lengths}
        n = sess.run(single_model.correct_num, feed_dict=feed_dict)
        count += np.sum(n)
    end_time = time.time()
    print('Testing Time:{:.2f}'.format(end_time - start_time))
    print(count*1.0/len(text_test))

Testing...
INFO:tensorflow:Restoring parameters from ckpt_cnn_pretrained_emb/cnn.ckpt
Parameters restored


ValueError: Cannot feed value of shape (1, 20, 100) for Tensor 'test/Model/Placeholder_3:0', which has shape '(1, 22, 100)'

In [25]:
print(count*1.0/len(text_test))

0.7275894836870447


In [23]:
sess_lm.close()